# [HW2] Topic Modeling
1. Crawling News
2. Preprocessing
3. Build Term-Document Matrix
4. Topic modeling
5. Visualization

```
🔥 이번 시간에는 Topic Modeling를 직접 크롤링한 뉴스 데이터에 대해서 수행해보는 시간을 갖겠습니다. 

먼저 네이버에서 뉴스 기사를 간단하게 크롤링합니다.
기본적인 전처리 이후 Term-document Matrix를 만들고 이를 non-negative factorization을 이용해 행렬 분해를 하여 Topic modeling을 수행합니다.

t-distributed stochastic neighbor embedding(T-SNE) 기법을 이용해 Topic별 시각화를 진행합니다.
```

In [1]:
!pip install newspaper3k
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 211 kB 5.0 MB/s 
     |████████████████████████████████| 7.4 MB 40.5 MB/s 
     |████████████████████████████████| 81 kB 2.7 MB/s 
     |████████████████████████████████| 93 kB 111 kB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13553 sha256=3fe16792bcc77918b9ef0c8bc80501fc7ce1e2674a0bec45d0b470a6c229bd4a
  Stored in directory: /root/.cache/pip/wheels/df/67/41/faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3357 sha256=6ecde599f8a239ed6361f31c88d025731fcc9d04446ee6d914b309eac6978ff2
  Stored in directory: /root/.cache/pip/wheels/7f/d4/8f/6e2ca54744c9d7292d88ddb8d42876bcdab5e6d84a21c10346
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398404 sha256=bd766eba81da50dfb775d7d95c38a2c59a79ab2abd7fff749

In [2]:
# 크롤링에 필요한 패키지 설치
from bs4 import BeautifulSoup
from newspaper import Article
from time import sleep
from time import time
from dateutil.relativedelta import relativedelta
from datetime import datetime
from multiprocessing import Pool
import json
import requests
import re
import sys

```
💡 Crawling(크롤링)이란?

크롤링은 웹 페이지에서 필요한 데이터를 추출해내는 작업을 말합니다.
이번 시간에는 정적 페이지인 네이버의 뉴스 신문 기사 웹페이지를 크롤링합니다.

HTML은 설명되어 있는 자료가 많기 때문에 생략하도록 하겠습니다.
HTML 구조 파악 및 태그에 대한 설명은 아래 참고자료를 살펴봐주세요 !
```

참고: [위키피디아: 정적페이지](https://ko.wikipedia.org/wiki/%EC%A0%95%EC%A0%81_%EC%9B%B9_%ED%8E%98%EC%9D%B4%EC%A7%80)

참고: [생활코딩: HTML](https://opentutorials.org/course/2039)

In [3]:
def crawl_news(query: str=None, crawl_num: int=1000, workers: int=4):
    '''뉴스 기사 텍스트가 담긴 list를 반환합니다.

    Keyword arguments:
    query -- 검색어 (default None)
    crawl_num -- 수집할 뉴스 기사의 개수 (defualt 1000)
    workers -- multi-processing시 사용할 thread의 개수 (default 4)
    '''

    url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'
    articleList = []
    crawled_url = set()
    keyboard_interrupt = False
    t = time()
    idx = 0
    page = 1

    
    # 서버에 url 요청의 결과를 선언
    res = requests.get(url.format(query))
    sleep(0.5)
    # res를 parsing할 parser를 선언
    bs = BeautifulSoup(res.text, 'html.parser')
    
    with Pool(workers) as p:
        while idx < crawl_num:            
            table = bs.find('ul', {'class': 'list_news'})
            li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
            area_list = [li.find('div', {'class':'news_area'}) for li in li_list]
            a_list = [area.find('a', {'class':'news_tit'}) for area in area_list]
            
            for n in a_list[:min(len(a_list), crawl_num-idx)]:
                articleList.append(n.get('title'))
                idx += 1
            page += 1

            pages = bs.find('div', {'class': 'sc_page_inner'})
            next_page_url = [p for p in pages.find_all('a') if p.text == str(page)][0].get('href')

            req = requests.get('https://search.naver.com/search.naver' + next_page_url)
            bs = BeautifulSoup(req.text, 'html.parser')
    return articleList

```
🔥 이제 '구글'이라는 이름으로 뉴스 기사 1000개의 제목을 크롤링하겠습니다.
```

In [4]:
query = '구글'

articleList = crawl_news(query)

In [5]:
articleList[:10]

['‘3나노 전쟁 본격화’ 구글, 대만 TSMC 대신 삼성전자 택했다',
 "사무실 출근하라면서 '사내 확진자 ○명' 매일 공지…구글 직원들 거부반응",
 '넥슨 히트2, 구글 매출 3위...단기 흥행 성공',
 '구글이 뉴욕캠퍼스에 레고방 만든 이유는?',
 'K스타트업 동행자 자처하는 빅테크…구글, AWS 이어 MS도 본격 지원',
 '이노스, 23년형 구글 OS 탑재 QLED 스마트TV 출시 예정',
 '삼성전자-구글, 픽셀8용 3세대 텐서 프로세서 개발 중',
 '구글, 크로스 디바이스 앱 개발 환경 확산 시동...SDK 공개',
 '야당, 구글·넷플릭스 겨냥 법개정 속도···ICT 손놓은 여당',
 '넥슨 \'히트2\' 구글 매출 2위 … "리니지M 섯거라!"']

```
🔥 태거(tagger)를 이용해 한글 명사와 알파벳만을 추출해서 term-document matrix (tdm)을 만들겠습니다.

태거(tagger)는 tokenization에서 조금 더 자세히 다루도록 하겠습니다.
```

참고: [konlpy: morph analyzer](https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/)

## Preprocessing

In [6]:
from konlpy.tag import Okt
from collections import Counter
import json

# Okt 형태소 분석기 선언
t = Okt()

words_list_ = []
vocab = Counter()
tag_set = set(['Noun', 'Alpha'])
stopwords = set(['글자'])

for i, article in enumerate(articleList):
    if i % 100 == 0:
        print(i)
    
    # tagger를 이용한 품사 태깅
    words = t.pos(article, norm=True, stem=True)

    ############################ ANSWER HERE ################################
    # TODO: 다음의 조건을 만족하는 단어의 리스트를 완성하세요.
    # 조건 1: 명사와 알파벳 tag를 가진 단어
    # 조건 2: 철자 길이가 2이상인 단어 
    # 조건 3: stopwords에 포함되지 않는 단어
    #########################################################################
    words = [w for w, t in words if t in tag_set and len(w) > 1 and w not in stopwords]        

    vocab.update(words)
    words_list_.append((words, article))
    
vocab = sorted([w for w, freq in vocab.most_common(10000)])
word2id = {w: i for i, w in enumerate(vocab)}
words_list = []
for words, article in words_list_:
    words = [w for w in words if w in word2id]
    if len(words) > 10:
        words_list.append((words, article))
        
del words_list_

0
100
200
300
400
500
600
700
800
900


## Build document-term matrix

```
🔥 이제 document-term matrix를 만들어보겠습니다.
document-term matrix는 (문서 개수 x 단어 개수)의 Matrix입니다.
```

참고: [Document-Term Matrix](https://wikidocs.net/24559)

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np

dtm = np.zeros((len(words_list), len(vocab)), dtype=np.float32)
for i, (words, article) in enumerate(words_list):
    for word in words:
        dtm[i, word2id[word]] += 1
        
dtm = TfidfTransformer().fit_transform(dtm)

```
🔥 document-term matrix를 non-negative factorization(NMF)을 이용해 행렬 분해를 해보겠습니다.

💡 Non-negative Factorization이란?

NMF는 주어진 행렬 non-negative matrix X를 non-negative matrix W와 H로 행렬 분해하는 알고리즘입니다.
이어지는 코드를 통해 W와 H의 의미에 대해 파악해봅시다.
```
참고: [Non-negative Matrix Factorization](https://angeloyeo.github.io/2020/10/15/NMF.html)

## Topic modeling

In [8]:
# Non-negative Matrix Factorization
from sklearn.decomposition import NMF

K=5
nmf = NMF(n_components=K, alpha=0.1)

```
🔥 sklearn의 NMF를 이용해 W와 H matrix를 구해봅시다.
W는 document length x K, H는 K x term length의 차원을 갖고 있습니다.
W의 하나의 row는 각각의 feature에 얼만큼의 가중치를 줄 지에 대한 weight입니다.
H의 하나의 row는 하나의 feature를 나타냅니다.

우선 하나의 Topic (H의 n번째 row)에 접근해서 해당 topic에 대해 값이 가장 높은 20개의 단어를 출력해보겠습니다.
```

In [9]:
W = nmf.fit_transform(dtm)
H = nmf.components_

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


```
🔥 우선 하나의 Topic (H의 n번째 row)에 접근해서 해당 topic에 대해 값이 가장 높은 20개의 단어를 출력해보겠습니다.
```

In [22]:

for k in range(K):
    print(f"{k}th topic")
    for index in H[k].argsort()[::-1][:20]:
        print(vocab[index], end=' ')
    print()

0th topic
AI 클라우드 제공 의료 스케 킷헬 기반 플랫폼 프린팅 융합 구글 바이오 국가 스마트 혁신 파트너 분야 민관 싱가포르 위해 
1th topic
매출 스토어 TOP 진입 리니지 크로니클 서머 컴투스 구글플레이 플레이 순위 넥슨 히트 레볼루션 세븐 넷마블 나이 상승세 달성 신작 
2th topic
결제 인앱 돋보기 IT 업계 구글 강제 방통위 정책 수수료 정산 애플 불만 콘텐츠 음악 변경 인상 스트리밍 저작권 방식 
3th topic
전자 갤럭시 삼성 테스트 글래스 행사 증권 주목 하나 공개 AR 구글 사용 프레임워크 옵트포머 파라미터 트랜스포머 최적화 하이퍼 마인드 
4th topic
인공 지능 경찰 사진 발진 탐지 의사 때문 아들 성기 신고 클라우드 구글 교수 최초 과정 학습 프라이버시 유민수 KAIST 


```
🔥 이번에는 W에서 하나의 Topic (W의 n번째 column)에 접근해서 해당 topic에 대해 값이 가장 높은 3개의 뉴스 기사 제목을 출력해보겠습니다.
```

In [23]:
for k in range(K):
    print(f"==={k}th topic===")
    for index in W[:, k].argsort()[::-1][:3]:
        print(words_list[index][1])
    print('\n')

===0th topic===
로킷헬스케어, 구글 클라우드 기반 AI·3D프린팅 융합 의료 플랫폼 제공
로킷헬스케어, 구글 클라우드 기반 AI·3D프린팅 융합 의료 플랫폼 제공
로킷헬스케어, 구글 클라우드 기반 AI‧3D 바이오프린팅 융합 의료 플랫폼 제공


===1th topic===
컴투스 신작 '서머너즈 워 크로니클', 구글플레이스토어 매출 TOP10 달성... 원스토어 매출도 상승세
컴투스 신작 '서머너즈 워 크로니클', 구글플레이스토어 매출 TOP10 달성... 원스토어 매출도 상승세
넥슨 기대작 '히트2' 구글 플레이 매출 3위 매출 TOP3 진입, '리니지M'과 '원신'도 업데이트로 순위 상승


===2th topic===
[인앱결제 A-Z] ④ 구글 신고 잇따라 접수…공정위, '인앱결제' 재검 [IT돋보기]
'구글 인앱결제' 음원 스트리밍 인상 원인…"저작권 정산방식 변경해야" [IT돋보기]
'구글 인앱결제' 음원 스트리밍 인상 원인…"저작권 정산방식 변경해야" [IT돋보기]


===3th topic===
하나증권 "8월 구글 AR글래스 테스트와 삼성전자 갤럭시 공개행사 주목"
하나증권 "8월 구글 AR글래스 테스트와 삼성전자 갤럭시 공개행사 주목"
하나증권 "8월 구글 AR글래스 테스트와 삼성전자 갤럭시 공개행사 주목"


===4th topic===
아들 성기 발진 때문에 의사에게 찍어 보낸 사진 탐지해 경찰에 신고한 '구글 클라우드' 인공지능
아들 성기 발진 때문에 의사에게 찍어 보낸 사진 탐지해 경찰에 신고한 '구글 클라우드' 인공지능
아들 성기 발진 때문에 의사에게 찍어 보낸 사진 탐지해 경찰에 신고한 '구글 클라우드' 인공지능




```
❓ 2번째 토픽에 대해 가장 높은 가중치를 갖는 제목 5개를 출력해볼까요?
```

In [27]:
#TODO
index_list = W[:, 1].argsort()[::-1][:5]
for index in index_list:
    print(words_list[index][1])

컴투스 신작 '서머너즈 워 크로니클', 구글플레이스토어 매출 TOP10 달성... 원스토어 매출도 상승세
컴투스 신작 '서머너즈 워 크로니클', 구글플레이스토어 매출 TOP10 달성... 원스토어 매출도 상승세
넥슨 기대작 '히트2' 구글 플레이 매출 3위 매출 TOP3 진입, '리니지M'과 '원신'도 업데이트로 순위 상승
넥슨 기대작 '히트2' 구글 플레이 매출 3위 매출 TOP3 진입, '리니지M'과 '원신'도 업데이트로 순위 상승
넷마블 '세븐나이츠 레볼루션' 구글 플레이 TOP 5 진입, 엔씨소프트 '리니지M' 매출 1위 탈환


```
🔥 이번에는 t-SNE를 이용해 Topic별 시각화를 진행해보겠습니다.

💡 t-SNE는 무엇인가요?

t-Stochastic Neighbor Embedding(t-SNE)은 고차원의 벡터를 
저차원(2~3차원) 벡터로 데이터간 구조적 특징을 유지하며 축소를 하는 방법 중 하나입니다.

주로 고차원 데이터의 시각화를 위해 사용됩니다.
```

참고: [lovit: t-SNE](https://lovit.github.io/nlp/representation/2018/09/28/tsne/#:~:text=t%2DSNE%20%EB%8A%94%20%EA%B3%A0%EC%B0%A8%EC%9B%90%EC%9D%98,%EC%9D%98%20%EC%A7%80%EB%8F%84%EB%A1%9C%20%ED%91%9C%ED%98%84%ED%95%A9%EB%8B%88%EB%8B%A4.)

참고: [ratsgo: t-SNE](https://ratsgo.github.io/machine%20learning/2017/04/28/tSNE/)

## Visualization

In [13]:
from sklearn.manifold import TSNE

# n_components = 차원 수
tsne = TSNE(n_components=2, init='pca', verbose=1)

# W matrix에 대해 t-sne를 수행합니다.
W2d = tsne.fit_transform(W)

# 각 뉴스 기사 제목마다 가중치가 가장 높은 topic을 저장합니다.
topicIndex = [v.argmax() for v in W]

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:986: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  FutureWarning,


[t-SNE] Computing 83 nearest neighbors...
[t-SNE] Indexed 84 samples in 0.000s...
[t-SNE] Computed neighbors for 84 samples in 0.002s...
[t-SNE] Computed conditional probabilities for sample 84 / 84
[t-SNE] Mean sigma: 0.098523
[t-SNE] KL divergence after 250 iterations with early exaggeration: 50.152016
[t-SNE] KL divergence after 1000 iterations: -0.287082


In [14]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
output_notebook()

# 사용할 툴들
tools_to_show = 'hover,box_zoom,pan,save,reset,wheel_zoom'
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article for words, article in words_list],
    'topic': [str(i) for i in topicIndex],  # 토픽 번호
    'color': [Category20[K][i] for i in topicIndex]
})
p.circle(
    'x', 'y',
    source=source,
    legend='topic',
    color='color'
)

# interaction
p.legend.location = "top_left"
hover = p.select({'type': HoverTool})
hover.tooltips = [("Topic", "@topic"), ('id', '@id'), ("Article", "@document")]
hover.mode = 'mouse'

show(p)